<a href="https://colab.research.google.com/github/SohamRatnaparkhi/BrainTumorDetection/blob/main/training_model_g_colab_SUCCESFULL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
from PIL import Image

In [12]:
tf.test.gpu_device_name()

'/device:GPU:0'

# Data managing

Unzipping data

In [ ]:
!unzip "/content/brain_tumor_dataset.zip"

Declaring Constants

In [5]:
SIZE = 64
path = r"/content/brain_tumor_dataset"

In [6]:
import os

In [7]:
len(list(os.listdir(path + "/" + "yes")))

1500

Renaming the dataset in a better format for future use
> **Note:** The code below should be run only once and not after that.  

In [8]:
import os

# path = r"D:\College\Capstone Project 2\data\brain_tumor_dataset"
path_items = os.listdir(path)
path_items

['yes', 'pred', 'no', 'Br35H-Mask-RCNN']

In [9]:
import os

# path = r"D:\College\Capstone Project 2\data\brain_tumor_dataset"
path_items = os.listdir(path)

yes_tumor_path = path + "/" + path_items[0]
yes_tumor = os.listdir(yes_tumor_path)
no_tumor_path = path + "/" + path_items[2]
no_tumor = os.listdir(no_tumor_path)

# try:
#     y = 1
#     for ytp in yes_tumor:
#         os.rename(src = yes_tumor_path + "/" + ytp, dst = yes_tumor_path + "/" + f"yes.{y}.jpg")
#         y += 1
        
#     n = 1
#     for ntp in no_tumor:
    
#         os.rename(src = no_tumor_path + "/" + ntp, dst = no_tumor_path + "/" + f"no.{n}.jpg")
#         n += 1
# except Exception as e:
#     print(e)



In [10]:
len(os.listdir(yes_tumor_path)), len(os.listdir(no_tumor_path))

(1500, 1500)

Resizing

Creating a `data` - `label` pair for `images` and its `type`

In [11]:
from PIL import Image
def resize_save(path, size = SIZE):
    try:
        im = Image.open(path)
        im = im.resize((size, size))
        im.save(path)
    except:
        print(path)

# Resize yes_tumor images
for p in yes_tumor:
    path = yes_tumor_path+"/"+p
    # resize_save(path)
    try:
        im = Image.open(path)
        im = im.resize((64,64))
        im.save(path)
    except:
        print(path)
        break

# Resize no_tumor
for p in no_tumor:
    path = no_tumor_path+"/"+p
    # resize_save(path)
    try:
        im = Image.open(path)
        im = im.resize((64, 64))
        im.save(path)
    except:
        print(path)
        break

In [13]:
len(os.listdir(yes_tumor_path)), len(os.listdir(no_tumor_path))

(1500, 1500)

In [14]:
n, y = 0, 0

In [15]:
no_tumor[0]

'no.1176.jpg'

In [60]:
data = []
label = []

# For no tumor 
for image in no_tumor:
    path = no_tumor_path + "/" + image
    image = cv.imread(path)
    # r_image = cv.resize(src = image, dsize = (SIZE, SIZE))

    data.append(image)
    label.append(0)

In [61]:
len(os.listdir(yes_tumor_path)), len(os.listdir(no_tumor_path))

(1500, 1500)

In [62]:
len(data) 

1500

In [63]:
#for yes tumor - 1
for image in yes_tumor:
    path = yes_tumor_path + "/" + image
    image = cv.imread(path)
    # r_image = cv.resize(src = image, dsize = (SIZE, SIZE))

    data.append(image)
    label.append(1)

In [64]:
len(data)

3000

OneHotEncoding the labels
> yes_tumour -> 1

> no_tumour -> 0

In [65]:
import numpy as np
data = np.array(data) / 255.0
label = np.array(label)

data.shape, label.shape

((3000, 64, 64, 3), (3000,))

In [66]:
# for i in range(3000):
#   if label[i] == "yes":
#     label[i] = 1
#   else:
#     label[i] = 0
label

array([0, 0, 0, ..., 1, 1, 1])

In [67]:
data.shape, label.shape

((3000, 64, 64, 3), (3000,))

In [68]:
from sklearn.model_selection import train_test_split as tts

x_train, x_test, y_train, y_test = tts(data, label, test_size = 0.20, random_state = 0)

In [69]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2400, 64, 64, 3), (600, 64, 64, 3), (2400,), (600,))

In [74]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((2400, 64, 64, 3), (2400,), (600, 64, 64, 3), (600,))

# Model Training

In [25]:
import tensorflow as tf

In [71]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(32, kernel_size = 3, activation = "relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

model.add(tf.keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", kernel_initializer='he_uniform'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

model.add(tf.keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", kernel_initializer='he_uniform'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(64, activation = "relu"))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation = "sigmoid"))

model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics = ["accuracy"])


model.fit(x_train, y_train, batch_size = 16, epochs = 10, validation_data = (x_test, y_test))

model.save("Brain_tumor_model_1.h5")

Epoch 1/10
150/150 [==============================] - 14s 15ms/step - loss: 0.5303 - accuracy: 0.7375 - val_loss: 0.4101 - val_accuracy: 0.8033
Epoch 2/10
150/150 [==============================] - 2s 13ms/step - loss: 0.3868 - accuracy: 0.8354 - val_loss: 0.3777 - val_accuracy: 0.8383
Epoch 3/10
150/150 [==============================] - 2s 13ms/step - loss: 0.3057 - accuracy: 0.8771 - val_loss: 0.2564 - val_accuracy: 0.8933
Epoch 4/10
150/150 [==============================] - 2s 13ms/step - loss: 0.2294 - accuracy: 0.9096 - val_loss: 0.2553 - val_accuracy: 0.8833
Epoch 5/10
150/150 [==============================] - 2s 11ms/step - loss: 0.1476 - accuracy: 0.9492 - val_loss: 0.1717 - val_accuracy: 0.9400
Epoch 6/10
150/150 [==============================] - 2s 12ms/step - loss: 0.0996 - accuracy: 0.9688 - val_loss: 0.1563 - val_accuracy: 0.9417
Epoch 7/10
150/150 [==============================] - 2s 12ms/step - loss: 0.0604 - accuracy: 0.9804 - val_loss: 0.1771 - val_accuracy: 0.943

In [72]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 31, 31, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 12, 12, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 6, 6, 64)        

In [76]:
model_2 = tf.keras.models.Sequential()

model_2.add(tf.keras.layers.Conv2D(32, kernel_size = 3, activation = "relu"))
model_2.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

model_2.add(tf.keras.layers.Conv2D(32, kernel_size = 3, activation = "relu", kernel_initializer='he_uniform'))
model_2.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

model_2.add(tf.keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", kernel_initializer='he_uniform'))
model_2.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

model_2.add(tf.keras.layers.Flatten())

model_2.add(tf.keras.layers.Dense(64, activation = "relu"))
model_2.add(tf.keras.layers.Dropout(0.5))

model_2.add(tf.keras.layers.Dense(1, activation = "sigmoid"))

model_2.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics = ["accuracy"])


history_2 = model_2.fit(x_train, y_train, batch_size = 16, epochs = 9, validation_data = (x_test, y_test))

model_2.save("Brain_tumor_model_2.h5")

Epoch 1/9
150/150 [==============================] - 3s 14ms/step - loss: 0.5327 - accuracy: 0.7396 - val_loss: 0.4449 - val_accuracy: 0.7950
Epoch 2/9
150/150 [==============================] - 2s 13ms/step - loss: 0.3958 - accuracy: 0.8317 - val_loss: 0.3368 - val_accuracy: 0.8533
Epoch 3/9
150/150 [==============================] - 2s 12ms/step - loss: 0.3028 - accuracy: 0.8746 - val_loss: 0.2824 - val_accuracy: 0.8717
Epoch 4/9
150/150 [==============================] - 2s 13ms/step - loss: 0.2175 - accuracy: 0.9246 - val_loss: 0.1905 - val_accuracy: 0.9300
Epoch 5/9
150/150 [==============================] - 2s 12ms/step - loss: 0.1351 - accuracy: 0.9558 - val_loss: 0.1278 - val_accuracy: 0.9600
Epoch 6/9
150/150 [==============================] - 2s 13ms/step - loss: 0.1107 - accuracy: 0.9592 - val_loss: 0.1425 - val_accuracy: 0.9483
Epoch 7/9
150/150 [==============================] - 2s 11ms/step - loss: 0.0656 - accuracy: 0.9821 - val_loss: 0.2289 - val_accuracy: 0.9200
Epoch 

In [77]:
model_3 = tf.keras.models.Sequential()

model_3.add(tf.keras.layers.Conv2D(64, kernel_size = 3, activation = "relu"))
model_3.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

model_3.add(tf.keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", kernel_initializer='he_uniform'))
model_3.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

model_3.add(tf.keras.layers.Conv2D(64, kernel_size = 3, activation = "relu", kernel_initializer='he_uniform'))
model_3.add(tf.keras.layers.MaxPooling2D(pool_size = (2,2)))

model_3.add(tf.keras.layers.Flatten())

model_3.add(tf.keras.layers.Dense(64, activation = "relu"))
model_3.add(tf.keras.layers.Dropout(0.5))

model_3.add(tf.keras.layers.Dense(1, activation = "sigmoid"))

model_3.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics = ["accuracy"])


history_3 = model_3.fit(x_train, y_train, batch_size = 16, epochs = 9, validation_data = (x_test, y_test))

model_3.save("Brain_tumor_model_3.h5")

Epoch 1/9
150/150 [==============================] - 3s 17ms/step - loss: 0.5400 - accuracy: 0.7312 - val_loss: 0.4269 - val_accuracy: 0.8150
Epoch 2/9
150/150 [==============================] - 2s 14ms/step - loss: 0.4082 - accuracy: 0.8192 - val_loss: 0.3374 - val_accuracy: 0.8717
Epoch 3/9
150/150 [==============================] - 2s 14ms/step - loss: 0.3074 - accuracy: 0.8800 - val_loss: 0.2712 - val_accuracy: 0.8850
Epoch 4/9
150/150 [==============================] - 2s 16ms/step - loss: 0.2260 - accuracy: 0.9150 - val_loss: 0.3344 - val_accuracy: 0.8567
Epoch 5/9
150/150 [==============================] - 2s 15ms/step - loss: 0.1631 - accuracy: 0.9458 - val_loss: 0.1668 - val_accuracy: 0.9400
Epoch 6/9
150/150 [==============================] - 2s 15ms/step - loss: 0.1021 - accuracy: 0.9629 - val_loss: 0.1380 - val_accuracy: 0.9550
Epoch 7/9
150/150 [==============================] - 2s 15ms/step - loss: 0.0730 - accuracy: 0.9762 - val_loss: 0.1169 - val_accuracy: 0.9550
Epoch 